In [18]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

In [19]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [20]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [21]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../../data/preprocessed_v2/clear_augmentation_v2.csv') # processed된 파일까지 경로 
TEST_DIR = os.path.join(BASE_DIR, '../../data/raw/test.csv')
train_name = os.path.splitext(os.path.basename(DATA_DIR))[0]  # processed된 파일 이름 추출
OUTPUT_DIR = os.path.join(BASE_DIR, f'../../output/{train_name}') # 해당 파일 이름으로 output 폴더 생성

In [22]:
# mapping: 생활문화(0) , 스포츠(1) , 정치(2) , 사회(3) , IT과학(4) , 경제(5) , 세계(6)
data = pd.read_csv(DATA_DIR)

mapping = {
    0: '생활문화',
    1: '스포츠',
    2: '정치',
    3: '사회',
    4: 'IT과학',
    5: '경제',
    6: '세계'
}

data['topic'] = data['target'].map(mapping)
data

,ID,text,target,topic
0,ynat-v1_train_00000,정의구현사제단 시국미사 미사용기간 단기간에 종결될 수 있다,4,IT과학
1,ynat-v1_train_00002,"김정일, 자주, 통일, 새, 열, 나가야, 보수",2,정치
2,ynat-v1_train_00004,대통령 선거를 앞두고 정치권 비방 댓글 단에 대한 감시가 강화되고 있다.,6,세계
3,ynat-v1_train_00006,광주-기아챔피언스필드에서 열릴 예정이던 롯데-KIA 경기가 우천으로 취소됐다.,1,스포츠
4,ynat-v1_train_00007,아포짓 스파이커 알렉스와 레오나르도 가빈이 이끄는 한국전력 빅스톰이 우리캐피탈 위비...,4,IT과학
...,...,...,...,...
6049,augmentation,5 . 게임 산업의 괄목할 성장과 사회적 영향,4,IT과학
6050,augmentation,이제 특이하고 매적인 문화 뉴스 제 목 6개를 만들어볼 차례야,0,생활문화
6051,augmentation,1 . 축구 월드컵 예선 한 국 팀 선전 분석,1,스포츠
6052,augmentation,2 . 신흥 강호 로 부상 하고 있는 한국 배구 리그,1,스포츠


In [23]:
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=SEED)

In [30]:
t = dataset_train
v = dataset_valid
v

,ID,text,target,topic
5621,augmentation,"3 기후 변화, 북극곰 멸종 위기 심화",0,생활문화
5592,augmentation,"답 서울 공유자전신 '띵동' 타자, 교통혁명은 이미 시작되었다",4,IT과학
2831,augmentation,"5. 답 유명 유튜버, 저작권 침르로 논란",4,IT과학
3956,augmentation,"5G 시대 도래, 초고속 무선 인터넷의 미래",4,IT과학
5417,augmentation,"1'. 코로나 예방, 스마트 마스크 등장",4,IT과학
...,...,...,...,...
5714,augmentation,"BTS, 빌보드 차트 또다시 석권할 듯",0,생활문화
5020,augmentation,"BTS, 그래미상 후보 지명...한국 음악 역사 새로 써",0,생활문화
1989,ynat-v1_train_02122,朴대통령 13일 북핵 담화·회견…경제법안 처리도 호소,2,정치
2883,augmentation,"5 우주여행, 내년 일반인 탑승 시작",4,IT과학


In [28]:
data['topic'].value_counts()

topic
IT과학    1600
생활문화    1346
사회       908
경제       724
정치       551
스포츠      471
세계       454
Name: count, dtype: int64

In [31]:
t['topic'].value_counts()

topic
IT과학    1274
생활문화    1050
사회       738
경제       599
정치       431
세계       380
스포츠      371
Name: count, dtype: int64

In [29]:
v['topic'].value_counts()

topic
IT과학    326
생활문화    296
사회      170
경제      125
정치      120
스포츠     100
세계       74
Name: count, dtype: int64

In [32]:
c = pd.read_csv('../../data/preprocessed_v2/clear_augmentation_v2.csv')
c

,ID,text,target
0,ynat-v1_train_00000,정의구현사제단 시국미사 미사용기간 단기간에 종결될 수 있다,4
1,ynat-v1_train_00002,"김정일, 자주, 통일, 새, 열, 나가야, 보수",2
2,ynat-v1_train_00004,대통령 선거를 앞두고 정치권 비방 댓글 단에 대한 감시가 강화되고 있다.,6
3,ynat-v1_train_00006,광주-기아챔피언스필드에서 열릴 예정이던 롯데-KIA 경기가 우천으로 취소됐다.,1
4,ynat-v1_train_00007,아포짓 스파이커 알렉스와 레오나르도 가빈이 이끄는 한국전력 빅스톰이 우리캐피탈 위비...,4
...,...,...,...
6049,augmentation,5 . 게임 산업의 괄목할 성장과 사회적 영향,4
6050,augmentation,이제 특이하고 매적인 문화 뉴스 제 목 6개를 만들어볼 차례야,0
6051,augmentation,1 . 축구 월드컵 예선 한 국 팀 선전 분석,1
6052,augmentation,2 . 신흥 강호 로 부상 하고 있는 한국 배구 리그,1


In [34]:
c['topic'] = c['target'].map(mapping)
c['topic'].value_counts()

topic
IT과학    1600
생활문화    1346
사회       908
경제       724
정치       551
스포츠      471
세계       454
Name: count, dtype: int64

In [36]:
c[c['topic'] == 'IT과학'].head(50)

,ID,text,target,topic
0,ynat-v1_train_00000,정의구현사제단 시국미사 미사용기간 단기간에 종결될 수 있다,4,IT과학
4,ynat-v1_train_00007,아포짓 스파이커 알렉스와 레오나르도 가빈이 이끄는 한국전력 빅스톰이 우리캐피탈 위비...,4,IT과학
12,ynat-v1_train_00027,전자서명 인증서 유효기간이 지나면 대체불가토큰(NFT) 끝,4,IT과학
21,ynat-v1_train_00052,주간지 위클리 스탠더드가 한국에서 얼마나 벌까요?,4,IT과학
23,ynat-v1_train_00054,감사합니다. 즐거운 시간 보내셨기를 바랍니다. *개* 카메라와 함께 즐거운 추억을 ...,4,IT과학
41,ynat-v1_train_00097,종이로도 삼겹살* 중공대사증후군* 델타변이* 예방접종 가능,4,IT과학
46,ynat-v1_train_00110,식물의 광합성을 모방한 스크럼 공장이 개발되고 있다.,4,IT과학
48,ynat-v1_train_00115,김두겸 울산광역시장(최병렬 전 한나라당 대표 보좌관 출신),4,IT과학
59,ynat-v1_train_00138,"삼성이 빅스를 개발하는 데 걸린 시간은 약 12개월이며, 서울이 최초이자 최초다. ...",4,IT과학
70,ynat-v1_train_00180,삼성 갤럭시 *용 ** 테스트 고객 무제한 요금제 테스트그룹 무차별 공격(DDoS)...,4,IT과학


In [38]:
# Lnoise_Augmentation_recover_4981_v1.csv
L_v1 = pd.read_csv('../../data/preprocessed_v2/Lnoise_Augmentation_recover_4981_v1.csv')
L_v2 = pd.read_csv('../../data/preprocessed_v2/Lnoise_Augmentation_recover_4981_v2.csv')

L_v1['topic'] = L_v1['target'].map(mapping)
L_v2['topic'] = L_v2['target'].map(mapping)

L_v1['topic'].value_counts()

topic
IT과학    1450
생활문화    1154
사회       776
경제       588
정치       378
스포츠      318
세계       317
Name: count, dtype: int64

In [39]:
L_v2['topic'].value_counts()

topic
IT과학    1453
생활문화    1194
사회       748
경제       590
정치       383
스포츠      317
세계       296
Name: count, dtype: int64